In [7]:
import librosa
import os
import soundfile as sf

def adjust_sample_rate(audio_path, target_sr=4000):
    # Cargar el archivo de audio
    y, sr = librosa.load(audio_path, sr=None)

    # Resamplear el audio
    y_resampled = librosa.resample(y, orig_sr=sr, target_sr=target_sr)

    return y_resampled, target_sr

# Directorio donde se encuentran los archivos de audio
input_dir = './audio_and_txt_files'

# Directorio donde se guardarán los archivos de audio con la frecuencia de muestreo ajustada
output_dir = "./Audios"

# Iterar sobre los archivos de audio en el directorio de entrada
for file_name in os.listdir(input_dir):
    if file_name.endswith('.wav'): 
        input_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name)

        # Ajustar la frecuencia de muestreo
        y_resampled, sr_resampled = adjust_sample_rate(input_path)

        # Guardar el archivo de audio con la frecuencia de muestreo ajustada
        sf.write(output_path, y_resampled, sr_resampled)


In [9]:
from pydub import AudioSegment
import os

# Directorio donde se encuentran los archivos .txt
directorio_txt_ciclos = './Audios'
directorio_txt_titulos = './ICBHI_challenge_train_test.txt'

# Directorio raíz donde se guardarán los nuevos archivos .wav organizados por tipo y conjunto
directorio_salida_raiz = './AudiosOfficial'

tipos = {
    '00': 'normal',
    '10': 'crackle',
    '01': 'wheeze',
    '11': 'crackle_wheeze'
}

# Crear las carpetas de salida si no existen
for conjunto in ['train', 'test']:
    for tipo_codigo, tipo_nombre in tipos.items():
        tipo_carpeta = os.path.join(directorio_salida_raiz, conjunto, tipo_nombre)
        os.makedirs(tipo_carpeta, exist_ok=True)

# Leer el archivo .txt con los títulos y carpetas
with open(directorio_txt_titulos, 'r') as titulos_file:
    titulos_lines = titulos_file.readlines()

# Recorrer los archivos en el directorio de los .txt de ciclos
for filename in os.listdir(directorio_txt_ciclos):
    if filename.endswith('.txt'):
        txt_path = os.path.join(directorio_txt_ciclos, filename)
        wav_filename = filename.replace('.txt', '.wav')

        # Encontrar la línea correspondiente en el archivo de títulos
        for titulo_line in titulos_lines:
            titulo_values = titulo_line.strip().split('\t')
            titulo_nombre = titulo_values[0]
            carpeta_conjunto = titulo_values[1]

            if titulo_nombre in wav_filename:
                break 

        # Verificar y crear la carpeta principal si no existe
        conjunto_carpeta = os.path.join(directorio_salida_raiz, carpeta_conjunto)
        os.makedirs(conjunto_carpeta, exist_ok=True)

        with open(txt_path, 'r') as txt_file:
            lines = txt_file.readlines()
            for index, line in enumerate(lines):
                values = line.strip().split('\t')
                inicio = float(values[0])
                fin = float(values[1])

                # Leer el archivo de audio original
                audio_path = os.path.join(directorio_txt_ciclos, wav_filename)
                audio = AudioSegment.from_wav(audio_path)

                # Extraer el segmento de audio correspondiente al ciclo
                ciclo_audio = audio[int(inicio * 1000):int(fin * 1000)]

                # Aplicar padding o recorte si es necesario
                target_duration = 4000  # Duración objetivo en milisegundos (4 segundos)
                if len(ciclo_audio) < target_duration:
                    print(f"Longitud de ciclo_audio: {len(ciclo_audio)}")
                    padding = AudioSegment.silent(duration=target_duration - len(ciclo_audio))
                    ciclo_audio += padding
                    print(f"Longitud de padding: {len(padding)}")
                    print(f"Longitud total después de sumar ciclo_audio y padding: {len(ciclo_audio)}")
                elif len(ciclo_audio) > target_duration:
                    ciclo_audio = ciclo_audio[:target_duration]
                    print(f"Longitud de ciclo_audio MAYOR: {len(ciclo_audio)}")
                else:
                    print(f"Longitud de ciclo_audio igual al target: {len(ciclo_audio)}")

                # Guardar el nuevo archivo de audio en la carpeta correspondiente
                tipo = values[2] + values[3]
                tipo_nombre_carpeta = tipos[tipo]
                nuevo_nombre = f"{tipo}_{index}_{wav_filename}"
                nuevo_path = os.path.join(directorio_salida_raiz, carpeta_conjunto, tipo_nombre_carpeta, nuevo_nombre)
                ciclo_audio.export(nuevo_path, format="wav")

                print(f"Archivo generado: {nuevo_path}")


Longitud de ciclo_audio: 543
Longitud de padding: 3457
Longitud total después de sumar ciclo_audio y padding: 4000
Archivo generado: ./AudiosOfficial\test\normal\00_0_101_1b1_Al_sc_Meditron.wav
Longitud de ciclo_audio: 1871
Longitud de padding: 2129
Longitud total después de sumar ciclo_audio y padding: 4000
Archivo generado: ./AudiosOfficial\test\normal\00_1_101_1b1_Al_sc_Meditron.wav
Longitud de ciclo_audio: 1443
Longitud de padding: 2557
Longitud total después de sumar ciclo_audio y padding: 4000
Archivo generado: ./AudiosOfficial\test\normal\00_2_101_1b1_Al_sc_Meditron.wav
Longitud de ciclo_audio: 1900
Longitud de padding: 2100
Longitud total después de sumar ciclo_audio y padding: 4000
Archivo generado: ./AudiosOfficial\test\normal\00_3_101_1b1_Al_sc_Meditron.wav
Longitud de ciclo_audio: 1728
Longitud de padding: 2272
Longitud total después de sumar ciclo_audio y padding: 4000
Archivo generado: ./AudiosOfficial\test\normal\00_4_101_1b1_Al_sc_Meditron.wav
Longitud de ciclo_audio: 1